In [ ]:
import pyper as pr
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental import preprocessing

In [54]:
r = pr.R(use_numpy='True', use_pandas='True')
r("load('data/signal_data_DF_to_python.RData')")
r("test_data <- as.data.frame(df_test_II_x)")
r("test_data_std <- as.data.frame(df_test_II_x_std)")
r("train_data <- as.data.frame(df_train_II_x)")
r("train_data_std <- as.data.frame(df_train_II_x_std)")
r("test_label <- as.data.frame(df_test_II_y)")
r("train_label <- as.data.frame(df_train_II_y)")

test_data = pd.DataFrame(r.get('test_data'))
test_data_std = pd.DataFrame(r.get('test_data_std'))
train_data = pd.DataFrame(r.get('train_data'))
train_data_std = pd.DataFrame(r.get('train_data_std'))
test_label = pd.DataFrame(r.get('test_label'))
train_label = pd.DataFrame(r.get('train_label'))

In [94]:
print("Train data shape:       ", train_data.shape)
print("Test data shape:        ", test_data.shape)
print("Train data (std) shape: ", train_data_std.shape)
print("Test data (std) shape:  ", test_data_std.shape)
print("Train label shape:      ", train_label.shape)
print("Test label shape:       ", test_label.shape)

Train data shape:        (2180, 4104)
Test data shape:         (932, 4104)
Train data (std) shape:  (2180, 4104)
Test data (std) shape:   (932, 4104)
Train label shape:       (2180, 1)
Test label shape:        (932, 1)


In [105]:
INPUT_DIM = 900
STEP = 4
cols = list(np.arange(train_data.shape[1]))
assert(train_data.shape[1]>=INPUT_DIM*STEP)

#new_cols = cols[0:INPUT_DIM*STEP:STEP]
new_cols = cols[:INPUT_DIM]

fin_train_data = train_data.iloc[:,new_cols]
fin_test_data = test_data.iloc[:,new_cols]

fin_train_data = np.array(fin_train_data)
fin_test_data = np.array(fin_test_data)
fin_train_label = train_label["df_train_II_y"].map({b'SR': 0, b'FA': 1})
fin_test_label = test_label["df_test_II_y"].map({b'SR': 0, b'FA': 1})

print("Train data shape:  ", fin_train_data.shape)
print("Test data shape:   ", fin_test_data.shape)
print("Train label shape: ", fin_train_label.shape)
print("Test label shape:  ", fin_test_label.shape)

Train data shape:   (2180, 900)
Test data shape:    (932, 900)
Train label shape:  (2180,)
Test label shape:   (932,)


In [106]:
fin_train_data = np.asarray(fin_train_data).astype('float32')
fin_test_data = np.asarray(fin_test_data).astype('float32')

fin_train_data2 = np.expand_dims(fin_train_data, axis=2)
fin_test_data2 = np.expand_dims(fin_test_data, axis=2)

print("Train data shape:  ", fin_train_data2.shape)
print("Test data shape:   ", fin_test_data2.shape)

Train data shape:   (2180, 900, 1)
Test data shape:    (932, 900, 1)


In [107]:
train = tf.data.Dataset.from_tensor_slices((fin_train_data2, fin_train_label))
test = tf.data.Dataset.from_tensor_slices((fin_test_data2, fin_test_label))

train = train.shuffle(len(train)).batch(2)
test = test.batch(16)

for d, l in train.take(1):
    print ("Data batch size:   ", d.shape)
    print ("Labels batch size: ", l.shape)

Data batch size:    (2, 900, 1)
Labels batch size:  (2,)


In [108]:
model = models.Sequential()
model.add(layers.BatchNormalization(input_shape=(900,1)))
model.add(layers.Conv1D(16,8))

model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(32,4))
model.add(layers.Conv1D(32,4))
model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(16, activation='relu'))

model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(32,2))
model.add(layers.Conv1D(32,2))
model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(8, activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(2))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 900, 1)            4         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 893, 16)           144       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 446, 16)           0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 446, 16)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 443, 32)           2080      
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 440, 32)           4128      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 220, 32)          

In [109]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

history = model.fit(train, epochs=200,
                       validation_data=(test))

Epoch 1/200
1090/1090 [==============================] - 9s 9ms/step - loss: 0.7001 - accuracy: 0.4950 - val_loss: 0.6932 - val_accuracy: 0.4979
Epoch 2/200
1090/1090 [==============================] - 9s 8ms/step - loss: 0.6965 - accuracy: 0.4885 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/200
1090/1090 [==============================] - 9s 8ms/step - loss: 0.6961 - accuracy: 0.5106 - val_loss: 0.6932 - val_accuracy: 0.4989
Epoch 4/200
1090/1090 [==============================] - 9s 8ms/step - loss: 0.6938 - accuracy: 0.4977 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 5/200
1090/1090 [==============================] - 9s 8ms/step - loss: 0.6938 - accuracy: 0.4927 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 6/200
1090/1090 [==============================] - 9s 9ms/step - loss: 0.6983 - accuracy: 0.5110 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/200
1090/1090 [==============================] - 9s 9ms/step - loss: 0.6938 - accuracy: 0.4876 - val_loss: 0.6932 - val_ac

KeyboardInterrupt: 